In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from functools import lru_cache
import re
from datetime import datetime
import os, json, time, tempfile

In [2]:
df_statements = pd.read_csv("data/annual_statements.csv")
all_tickers = df_statements['tic'].unique().tolist()
all_tickers[:10]

['AIR',
 '4165A',
 'ADCT.1',
 'IWKS',
 'ALO.2',
 'AEN.2',
 'AAL',
 '4267A',
 'CECE',
 'ARXX']

In [22]:
START = "2003-01-01"
# yfinance's `end` is exclusive; use Aug 1 to include all of July 2025
END   = "2025-08-01"
OUT_PATH = "data/monthly_returns.csv"
LOG_PATH = "data/monthly_returns.log.jsonl"
os.makedirs(os.path.dirname(OUT_PATH), exist_ok=True)

def normalize_for_yahoo(sym: str) -> list[str]:
    """
    Produce candidate Yahoo symbols from a Compustat-like ticker.
    e.g., 'BRK.B' -> ['BRK-B','BRK.B'], 'GKIN.1' -> ['GKIN','GKIN-1','GKIN.1']
    """
    s = str(sym).strip().upper()
    cands = [s]
    if "." in s:
        cands.append(s.replace(".", "-"))
        # drop trailing .<digits> like GKIN.1
        if re.search(r"\.\d+$", s):
            cands.append(re.sub(r"\.\d+$", "", s))
    if "/" in s:
        cands.append(s.replace("/", "-"))
    # de-dup while preserving order
    seen, out = set(), []
    for x in cands:
        if x and x not in seen:
            out.append(x); seen.add(x)
    return out

@lru_cache(maxsize=None)
def _download_monthly_adjclose(sym: str, start: str = START, end: str = END) -> pd.Series | None:
    """
    Try normalized candidates; return a clean monthly adjusted close Series or None.
    """
    for cand in normalize_for_yahoo(sym):
        try:
            df = yf.download(cand, start=start, end=end, interval="1mo",
                             auto_adjust=True, progress=False, keepna=False)
            if not df.empty:
                col = "Adj Close" if "Adj Close" in df.columns else "Close"
                px = df[col].dropna()
                if not px.empty:
                    px.index = px.index.to_period('M')
                    return px
        except Exception:
            # swallow and try next candidate
            pass
    return None

def _atomic_save_csv(df: pd.DataFrame, path: str):
    """Write CSV atomically to avoid partial files on interruption."""
    dir_ = os.path.dirname(path) or "."
    with tempfile.NamedTemporaryFile("w", dir=dir_, delete=False, suffix=".tmp") as tmp:
        tmp_path = tmp.name
        df.to_csv(tmp_path, index=True)
    os.replace(tmp_path, path)

def _append_log(event: dict, path: str = LOG_PATH):
    event = dict(event)
    event["ts"] = datetime.utcnow().isoformat(timespec="seconds") + "Z"
    with open(path, "a") as f:
        f.write(json.dumps(event) + "\n")

def monthly_returns_df(tickers: list[str],
                       start: str = START,
                       end: str = END,
                       min_non_na: int = 1,
                       as_period_index: bool = True,
                       out_path: str = OUT_PATH) -> pd.DataFrame:
    """
    Build monthly returns for tickers from Jan 2003 to Jul 2025.
    Checkpoints to CSV after each ticker, and resumes from that CSV if present.
    Columns are the ORIGINAL tickers you passed in.
    """
    # Master monthly grid
    full_pidx = pd.period_range("2003-01", "2025-07", freq="M")

    # Resume if an output file already exists
    if os.path.exists(out_path):
        prev = pd.read_csv(out_path, index_col=0)
        # Convert index back to PeriodIndex if it isn't already
        try:
            prev.index = pd.PeriodIndex(prev.index, freq="M")
        except Exception:
            # If index looks like timestamps, convert then to Period
            prev.index = pd.to_datetime(prev.index).to_period("M")
        # Align to our master grid
        prev = prev.reindex(full_pidx)
        done = set(prev.columns)
        df = prev
        _append_log({"event": "resume", "columns": sorted(list(done)), "n_cols": len(done)})
        print(f"Resuming from '{out_path}' with {len(done)} columns.")
    else:
        df = pd.DataFrame(index=full_pidx)
        done = set()
        print("Starting fresh.")

    skipped = []
    attempted = 0
    added = 0

    for t in tickers:
        if t in done:
            continue

        attempted += 1
        # simple backoff loop for transient rate-limit errors
        for attempt in range(4):  # up to 4 tries with backoff
            try:
                px = _download_monthly_adjclose(t, start=start, end=end)
                break
            except Exception as e:
                wait = 2 ** attempt
                _append_log({"event": "exception", "ticker": t, "attempt": attempt+1, "error": repr(e)})
                time.sleep(wait)
        else:
            px = None  # all attempts failed

        if px is None or len(px) <= 2:
            skipped.append(t)
            _append_log({"event": "skip", "ticker": t, "reason": "no_data_or_short"})
            continue

        # compute monthly returns; ensure Series
        rets = px.pct_change().dropna()
        if isinstance(rets, pd.DataFrame):
            rets = rets.squeeze()

        # column DF named by original ticker, aligned to master grid
        col_df = pd.DataFrame(rets)
        col_df.columns = [t]
        col_df = col_df.reindex(full_pidx)

        # Merge into our accumulating DF (preserving any existing columns)
        df[t] = col_df[t]
        added += 1
        done.add(t)

        # Optional prune ultra-sparse columns on the fly
        if min_non_na > 1 and df[t].count() < min_non_na:
            # remove and mark skipped if too sparse
            df.drop(columns=[t], inplace=True)
            done.discard(t)
            skipped.append(t)
            _append_log({"event": "drop_sparse", "ticker": t, "non_na": int(col_df[t].count())})
        else:
            # Checkpoint after each successful add
            _atomic_save_csv(df, out_path)
            _append_log({"event": "checkpoint", "ticker": t, "n_cols": int(df.shape[1])})
            print(f"Saved checkpoint with '{t}' (now {df.shape[1]} cols).")

    if not df.columns.tolist():
        print("No tickers returned data.")
        # keep index as requested
        if not as_period_index:
            out = df.copy()
            out.index = out.index.to_timestamp('M')
            return out
        return df

    # Final tidy: drop ultra-sparse columns globally if needed
    if min_non_na > 1:
        keep = df.columns[df.count() >= min_non_na]
        if len(keep) != df.shape[1]:
            dropped = sorted(set(df.columns) - set(keep))
            df = df[keep]
            _append_log({"event": "final_drop_sparse", "dropped": dropped})

    if skipped:
        print(f"Skipped (no/short data): {len(skipped)} → {sorted(skipped)[:20]}{' ...' if len(skipped)>20 else ''}")
    print(f"Built monthly returns for {df.shape[1]} tickers, {df.shape[0]} months. "
          f"(attempted: {attempted}, added: {added})")

    # Save final snapshot (idempotent)
    _atomic_save_csv(df, out_path)
    _append_log({"event": "final_save", "n_cols": int(df.shape[1])})

    # Return with desired index type
    if as_period_index:
        return df
    out = df.copy()
    out.index = out.index.to_timestamp('M')  # month-end timestamps
    return out

# -------- Example usage --------
# all_tickers = ["AAPL", "MSFT", "BRK.B", "GOOG", "ACEL.", "BFTC", "XOM", "BRK.A"]
df_mret = monthly_returns_df(all_tickers[18461:], out_path=OUT_PATH)
print(df_mret.tail())

Resuming from 'data/monthly_returns.csv' with 4462 columns.



1 Failed download:
['MYDR.1']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['MYDR-1']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['MYDR']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['OSN']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['HZNP']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['DRII']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2003-01-01 -> 2025-08-01)')


Saved checkpoint with 'EVTC' (now 4463 cols).



1 Failed download:
['PWON']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['5240B']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['TAIS']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['NCTX']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['CTNA']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['EMGCQ']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['ECYT']: YFPricesMissingError('possibly delisted; no price data found  (1mo 2003-01-01 -> 2025-08-01)')

1 Failed download:
['TXU1']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['LZENF']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['CPHB']: YFTzMissingError('possibly delisted; no timezone found')

1 Failed download:
['TPRFF']: YFTzMissingError('possibly delisted; no timezone found')



Saved checkpoint with 'SCHD' (now 4464 cols).


...

Saved checkpoint with 'SNOW' (now 4728 cols).


/var/folders/_d/chgk8nwx2wn005yx81lw4xl80000gn/T/ipykernel_93880/1093066496.py:134: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[t] = col_df[t]


Saved checkpoint with 'HYFM' (now 4729 cols).



1 Failed download:
['WISH']: YFTzMissingError('possibly delisted; no timezone found')


Skipped (no/short data): 682 → ['0074A', '0168A', '0182A', '0184A', '0188A', '0935B', '1684B', '2151B', '21ST10', '2793B', '2891B', '3312B', '3CIGW', '3JWCA', '3PNAM', '4934B', '5240B', '5305B', '5377B', '5445B'] ...
Built monthly returns for 4729 tickers, 271 months. (attempted: 949, added: 267)
              AIR    ADCT.1       AAL       ASA       PNW       PRG       ABT  \
2025-03 -0.138880 -0.198864 -0.264808  0.234740  0.039988 -0.062390 -0.038838   
2025-04 -0.045187  0.014184 -0.056872 -0.005309 -0.000735 -0.004235 -0.014323   
2025-05  0.148709  1.160839  0.146734  0.055704 -0.041500  0.093703  0.026377   
2025-06  0.120176 -0.132686 -0.016652 -0.001488 -0.009897  0.022441  0.018191   
2025-07  0.086059  0.033582  0.024064 -0.019639  0.012853  0.084838 -0.072201   

           SERV.1      WDDD     ACMTA  ...       APG       NVT     CSTPF  \
2025-03 -0.361820 -0.170000  0.000000  ... -0.089381 -0.131256 -0.040000   
2025-04  0.073043 -0.120482  0.204819  ...  0.057886  0.047501 

In [ ]:
def top_contributors(deciles_by_year, df_prices, decile=4, k=5, topn=10):
    acc = []
    by_year = deciles_by_year['bm']
    for Y, decmap in by_year.items():
        for t in decmap.get(decile, []):
            r1to5 = postformation_returns_5y_from_df(df_prices, t, Y)
            if len(r1to5) == 5 and np.isfinite(r1to5[k-1]):
                acc.append((Y, t, r1to5[k-1]))
    acc = sorted(acc, key=lambda x: (-abs(x[2]), x[0]))[:topn]
    return pd.DataFrame(acc, columns=["formation_year", "ticker", f"R{k}"])

df_bad = top_contributors(deciles_by_year, df_prices, decile=4, k=5, topn=15)
print(df_bad)

# Spot-check a ticker/year window
Y, t = int(df_bad.iloc[0]['formation_year']), df_bad.iloc[0]['ticker']
months = [pd.Period(f"{Y+4}-05","M")+i for i in range(12)]
print(df_prices[t].loc[months])  # raw

In [ ]:
df_prices = pd.read_csv("data/monthly_returns.csv")
bad_tickers = ['NST', 'TBL', 'CNT', 'CDRBQ', 'FRK']
df_prices = df_prices.drop(columns=[c for c in bad_tickers if c in df_prices.columns])
df_prices.to_csv("data/monthly_returns3.csv", index=True)

In [ ]:
def clean_df_prices(df):
    """Sanitize monthly decimal returns. Drop columns with too many bad rows."""
    dfc = df.copy()

    # Remove insane values (< -0.99, > +3.0)
    bad_mask = (dfc < -0.99) | (dfc > 3.0)
    dfc[bad_mask] = np.nan

    # Drop tickers with <150 valid months (e.g. delisted, garbage)
    min_obs = 150
    valid_counts = dfc.count()
    dfc = dfc.loc[:, valid_counts >= min_obs]

    # Optional: Drop tickers with any NaN in rolling 12-month windows
    bad_cols = []
    for col in dfc.columns:
        if dfc[col].isna().rolling(12).sum().min() > 0:  # no full 12-month window
            bad_cols.append(col)

    dfc = dfc.drop(columns=bad_cols)

    print(f"Cleaned df: {dfc.shape[1]} tickers kept (dropped {len(df.columns) - dfc.shape[1]})")
    return dfc

df_prices = pd.read_csv("data/monthly_returns3.csv", index_col=0)
df_prices.index = pd.PeriodIndex(df_prices.index, freq="M")
df_prices = clean_df_prices(df_prices)

df_prices.to_csv("data/monthly_returns4.csv", index=True)
